In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from nltk.tree import Tree
import svgling

import networkx as nx
from scipy import sparse

from IPython import embed

In [2]:
import pickle

save_dir = "D:\\MIT\\Courses\\2023-Spring\\6.8630-NLP\\Final Project\\LLM CGW\\All Parse Trees"

### Convert Tree to Graph

In [3]:
from nltk.tree.probabilistic import ProbabilisticTree

def get_node_structure(node, max_id):
    label = max_id+1
    max_id_temp = max_id+1
    
    children = []
    for i in range(len(node)):
        if type(node[i]) in [ProbabilisticTree, Tree]:
            child, max_id_temp = get_node_structure(node[i], max_id_temp)
            children.append(child)
        else:
            max_id_temp += 1
            children.append([max_id_temp])
            
    
    return [label, children], max_id_temp

In [4]:
def convert_graph(tree):
    node_structure, num_nodes = get_node_structure(tree, 0)
    
    queue = [node_structure]
    
    G = nx.DiGraph()
    
    while len(queue)>0:
        node = queue.pop(0)
        
        if len(node)>1:
            parent_id = node[0]
            children = node[1]
        
            for i in range(len(children)):
                child = children[i]
                child_id = child[0]
            
                G.add_edge(parent_id, child_id)
            
                queue.append(child)
        else: # leaf node, edge pointing to it is already added, so continue without any additional operations
            continue
    
    return G

### Graph-related Computation for Trees

In [5]:
def network_measures(G, model):
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    
    if model == 'PCFG':
        width = G.degree[2]
        depth = len(nx.dag_longest_path(G))-1
    elif model == 'Google':
        width = G.degree[1]-1
        depth = len(nx.dag_longest_path(G))
    else:
        width = G.degree[1]
        depth = len(nx.dag_longest_path(G))
    
    betweenness_centrality = dict(nx.betweenness_centrality(G.to_undirected()))
    betweenness_centrality = dict(sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True))
    max_betweenness = (list(betweenness_centrality.keys())[0], list(betweenness_centrality.values())[0])
    mean_betweenness = np.mean((np.array(list(betweenness_centrality.values()))))
    
    degree_centrality = dict(nx.degree_centrality(G.to_undirected()))
    degree_centrality = dict(sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True))
    max_degree = (list(degree_centrality.keys())[0], list(degree_centrality.values())[0])
    mean_degree = np.mean((np.array(list(degree_centrality.values()))))
    
    measures = {
        'num_nodes': num_nodes,
        'num_edges': num_edges,
        'width': width,
        'depth': depth,
        'max_betweenness': max_betweenness,
        'mean_betweenness': mean_betweenness,
        'max_degree_centrality': max_degree,
        'mean_degree_centrality': mean_degree
    }
    return measures

### Read Sentences

In [6]:
def read_file_lines(text_file):
    """
        reads the text_file
        returns a list of uncommented lines
    """
    lines = []
    with open(text_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                continue  # comment
            lines.append(line)
    return lines

In [7]:
base_sent = read_file_lines(save_dir + '\\Sentences\\base-set.sen')
adversarial_sent = read_file_lines(save_dir + '\\Sentences\\adversarial-set.sen')

## 1. Simple: be

In [8]:
id_1_base = [2, 4, 10]
id_1_adversarial = [1, 5, 11, 25, 27, 28, 47, 53, 56, 58, 86]

In [9]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_1 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_1_base)):
        tree = trees[id_1_base[idx]-1]
        base_trees_1[model].append(tree)

In [10]:
base_sent_1 = [base_sent[i-1] for i in id_1_base]

base_metrics_1 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_1:
    base_metrics_1['Sent'].append(sent)

for idx in range(len(base_sent_1)):
    for model in MODELS:
        tree = base_trees_1[model][idx]
        graph = convert_graph(tree)
        base_metrics_1[model].append(network_measures(graph, model))

In [13]:
print("1. Simple: be \n")
print("base \n")
for idx in range(len(base_sent_1)):
    print(idx)
    print(base_metrics_1['Sent'][idx])
    for metric in list(base_metrics_1['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_1['PCFG'][idx][metric], 
                                                          base_metrics_1['ChatGPT'][idx][metric], 
                                                          base_metrics_1['Google'][idx][metric]))

1. Simple: be 

base 

0
Arthur is the king .
  num_nodes: 
    PCFG: 21  ChatGPT: 14  Google: 17
  num_edges: 
    PCFG: 20  ChatGPT: 13  Google: 16
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 8  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: (9, 0.6105263157894737)  ChatGPT: (5, 0.6666666666666666)  Google: (1, 0.75)
  mean_betweenness: 
    PCFG: 0.21654135338345865  ChatGPT: 0.20146520146520144  Google: 0.18333333333333332
  max_degree_centrality: 
    PCFG: (2, 0.15000000000000002)  ChatGPT: (1, 0.23076923076923078)  Google: (1, 0.25)
  mean_degree_centrality: 
    PCFG: 0.09523809523809523  ChatGPT: 0.14285714285714285  Google: 0.11764705882352941
1
riding to Camelot is hard .
  num_nodes: 
    PCFG: 22  ChatGPT: 16  Google: 21
  num_edges: 
    PCFG: 21  ChatGPT: 15  Google: 20
  width: 
    PCFG: 3  ChatGPT: 2  Google: 3
  depth: 
    PCFG: 9  ChatGPT: 6  Google: 6
  max_betweenness: 
    PCFG: (4, 0.6571428571428573)  ChatGPT: (2, 0.8)  Google: (1, 0.

In [14]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_1 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_1_adversarial)):
        tree = trees[id_1_adversarial[idx]-1]
        adversarial_trees_1[model].append(tree)

In [15]:
adversarial_sent_1 = [adversarial_sent[i-1] for i in id_1_adversarial]

adversarial_metrics_1 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_1:
    adversarial_metrics_1['Sent'].append(sent)

for idx in range(len(adversarial_sent_1)):
    for model in MODELS:
        tree = adversarial_trees_1[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_1[model].append(network_measures(graph, model))

In [16]:
print("1. Simple: be \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_1)):
    print(idx)
    print(adversarial_metrics_1['Sent'][idx])
    for metric in list(adversarial_metrics_1['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_1['PCFG'][idx][metric], 
                                                          adversarial_metrics_1['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_1['Google'][idx][metric]))

1. Simple: be 

adversarial 

0
being is weary !
  num_nodes: 
    PCFG: 14  ChatGPT: 11  Google: 13
  num_edges: 
    PCFG: 13  ChatGPT: 10  Google: 12
  width: 
    PCFG: 3  ChatGPT: 1  Google: 3
  depth: 
    PCFG: 5  ChatGPT: 5  Google: 4
  max_betweenness: 
    PCFG: (2, 0.6923076923076923)  ChatGPT: (5, 0.8)  Google: (1, 0.8181818181818182)
  mean_betweenness: 
    PCFG: 0.2234432234432234  ChatGPT: 0.19191919191919193  Google: 0.20745920745920748
  max_degree_centrality: 
    PCFG: (2, 0.23076923076923078)  ChatGPT: (5, 0.4)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.14285714285714282  ChatGPT: 0.18181818181818185  Google: 0.15384615384615383
1
mountains are to England !
  num_nodes: 
    PCFG: 24  ChatGPT: 15  Google: 17
  num_edges: 
    PCFG: 23  ChatGPT: 14  Google: 16
  width: 
    PCFG: 3  ChatGPT: 2  Google: 3
  depth: 
    PCFG: 10  ChatGPT: 6  Google: 5
  max_betweenness: 
    PCFG: (8, 0.6561264822134387)  ChatGPT: (5, 0.7472527472527473)  

## 2. Simple: vi

In [17]:
id_2_base = [11, 24]
id_2_adversarial = [6, 19, 20, 22, 24, 42, 52, 57, 60, 61, 66, 69]

In [18]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_2 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_2_base)):
        tree = trees[id_2_base[idx]-1]
        base_trees_2[model].append(tree)

In [19]:
base_sent_2 = [base_sent[i-1] for i in id_2_base]

base_metrics_2 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_2:
    base_metrics_2['Sent'].append(sent)

for idx in range(len(base_sent_2)):
    for model in MODELS:
        tree = base_trees_2[model][idx]
        graph = convert_graph(tree)
        base_metrics_2[model].append(network_measures(graph, model))

In [20]:
print("2. Simple: vi \n")
print("base \n")
for idx in range(len(base_sent_2)):
    print(idx)
    print(base_metrics_2['Sent'][idx])
    for metric in list(base_metrics_2['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_2['PCFG'][idx][metric], 
                                                          base_metrics_2['ChatGPT'][idx][metric], 
                                                          base_metrics_2['Google'][idx][metric]))

2. Simple: vi 

base 

0
Guinevere might have known .
  num_nodes: 
    PCFG: 17  ChatGPT: 14  Google: 16
  num_edges: 
    PCFG: 16  ChatGPT: 13  Google: 15
  width: 
    PCFG: 3  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 6  ChatGPT: 5  Google: 4
  max_betweenness: 
    PCFG: (2, 0.6583333333333333)  ChatGPT: (5, 0.6666666666666666)  Google: (1, 0.8571428571428572)
  mean_betweenness: 
    PCFG: 0.19803921568627453  ChatGPT: 0.20146520146520144  Google: 0.17261904761904762
  max_degree_centrality: 
    PCFG: (8, 0.25)  ChatGPT: (1, 0.23076923076923078)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.11764705882352941  ChatGPT: 0.14285714285714285  Google: 0.125
1
Arthur and Guinevere migrate frequently .
  num_nodes: 
    PCFG: 25  ChatGPT: 16  Google: 20
  num_edges: 
    PCFG: 24  ChatGPT: 15  Google: 19
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 8  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: (3, 0.6920289855072463)  ChatGPT: (1,

In [21]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_2 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_2_adversarial)):
        tree = trees[id_2_adversarial[idx]-1]
        adversarial_trees_2[model].append(tree)

In [22]:
adversarial_sent_2 = [adversarial_sent[i-1] for i in id_2_adversarial]

adversarial_metrics_2 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_2:
    adversarial_metrics_2['Sent'].append(sent)

for idx in range(len(adversarial_sent_2)):
    for model in MODELS:
        tree = adversarial_trees_2[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_2[model].append(network_measures(graph, model))

In [23]:
print("2. Simple: vi \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_2)):
    print(idx)
    print(adversarial_metrics_2['Sent'][idx])
    for metric in list(adversarial_metrics_2['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_2['PCFG'][idx][metric], 
                                                          adversarial_metrics_2['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_2['Google'][idx][metric]))

2. Simple: vi 

adversarial 

0
the castle might have covered !
  num_nodes: 
    PCFG: 20  ChatGPT: 16  Google: 20
  num_edges: 
    PCFG: 19  ChatGPT: 15  Google: 19
  width: 
    PCFG: 3  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 6  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: (2, 0.6549707602339181)  ChatGPT: (1, 0.6285714285714287)  Google: (1, 0.8070175438596491)
  mean_betweenness: 
    PCFG: 0.18333333333333335  ChatGPT: 0.18571428571428572  Google: 0.1570175438596491
  max_degree_centrality: 
    PCFG: (11, 0.21052631578947367)  ChatGPT: (1, 0.2)  Google: (1, 0.2631578947368421)
  mean_degree_centrality: 
    PCFG: 0.09999999999999998  ChatGPT: 0.125  Google: 0.09999999999999998
1
Guinevere knows !
  num_nodes: 
    PCFG: 13  ChatGPT: 9  Google: 10
  num_edges: 
    PCFG: 12  ChatGPT: 8  Google: 9
  width: 
    PCFG: 3  ChatGPT: 2  Google: 2
  depth: 
    PCFG: 6  ChatGPT: 4  Google: 4
  max_betweenness: 
    PCFG: (2, 0.7121212121212122)  ChatGPT: (5, 0.714285714285714

## 3. Simple: vi with PP

In [28]:
id_3_base = [1, 12, 21]
id_3_adversarial = [10, 34, 43, 46, 48, 59, 77, 84]

In [29]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_3 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_3_base)):
        tree = trees[id_3_base[idx]-1]
        base_trees_3[model].append(tree)

In [30]:
base_sent_3 = [base_sent[i-1] for i in id_3_base]

base_metrics_3 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_3:
    base_metrics_3['Sent'].append(sent)

for idx in range(len(base_sent_3)):
    for model in MODELS:
        tree = base_trees_3[model][idx]
        graph = convert_graph(tree)
        base_metrics_3[model].append(network_measures(graph, model))

In [31]:
print("3. Simple: vi with PP \n")
print("base \n")
for idx in range(len(base_sent_3)):
    print(idx)
    print(base_metrics_3['Sent'][idx])
    for metric in list(base_metrics_3['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_3['PCFG'][idx][metric], 
                                                          base_metrics_3['ChatGPT'][idx][metric], 
                                                          base_metrics_3['Google'][idx][metric]))

3. Simple: vi with PP 

base 

0
Arthur rode to Camelot .
  num_nodes: 
    PCFG: 22  ChatGPT: 15  Google: 17
  num_edges: 
    PCFG: 21  ChatGPT: 14  Google: 16
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 10  ChatGPT: 6  Google: 5
  max_betweenness: 
    PCFG: (9, 0.6761904761904762)  ChatGPT: (5, 0.6593406593406594)  Google: (1, 0.75)
  mean_betweenness: 
    PCFG: 0.2158008658008658  ChatGPT: 0.20146520146520147  Google: 0.18333333333333332
  max_degree_centrality: 
    PCFG: (9, 0.19047619047619047)  ChatGPT: (1, 0.21428571428571427)  Google: (1, 0.25)
  mean_degree_centrality: 
    PCFG: 0.09090909090909091  ChatGPT: 0.1333333333333333  Google: 0.11764705882352941
1
Guinevere should be riding with Patsy .
  num_nodes: 
    PCFG: 27  ChatGPT: 21  Google: 23
  num_edges: 
    PCFG: 26  ChatGPT: 20  Google: 22
  width: 
    PCFG: 3  ChatGPT: 3  Google: 5
  depth: 
    PCFG: 10  ChatGPT: 8  Google: 5
  max_betweenness: 
    PCFG: (8, 0.6738461538461539)  ChatGPT: 

In [32]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_3 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_3_adversarial)):
        tree = trees[id_3_adversarial[idx]-1]
        adversarial_trees_3[model].append(tree)

In [33]:
adversarial_sent_3 = [adversarial_sent[i-1] for i in id_3_adversarial]

adversarial_metrics_3 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_3:
    adversarial_metrics_3['Sent'].append(sent)

for idx in range(len(adversarial_sent_3)):
    for model in MODELS:
        tree = adversarial_trees_3[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_3[model].append(network_measures(graph, model))

In [34]:
print("3. Simple: vi with PP \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_3)):
    print(idx)
    print(adversarial_metrics_3['Sent'][idx])
    for metric in list(adversarial_metrics_3['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_3['PCFG'][idx][metric], 
                                                          adversarial_metrics_3['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_3['Google'][idx][metric]))

3. Simple: vi with PP 

adversarial 

0
that horse might be growing to the Holy Grail .
  num_nodes: 
    PCFG: 34  ChatGPT: 26  Google: 34
  num_edges: 
    PCFG: 33  ChatGPT: 25  Google: 33
  width: 
    PCFG: 3  ChatGPT: 3  Google: 5
  depth: 
    PCFG: 10  ChatGPT: 7  Google: 6
  max_betweenness: 
    PCFG: (11, 0.6401515151515151)  ChatGPT: (9, 0.64)  Google: (1, 0.7651515151515151)
  mean_betweenness: 
    PCFG: 0.15786541889483066  ChatGPT: 0.14307692307692307  Google: 0.11408199643493762
  max_degree_centrality: 
    PCFG: (11, 0.12121212121212122)  ChatGPT: (9, 0.16)  Google: (1, 0.18181818181818182)
  mean_degree_centrality: 
    PCFG: 0.058823529411764705  ChatGPT: 0.07692307692307693  Google: 0.058823529411764705
1
Camelot will be covering to England .
  num_nodes: 
    PCFG: 27  ChatGPT: 18  Google: 23
  num_edges: 
    PCFG: 26  ChatGPT: 17  Google: 22
  width: 
    PCFG: 3  ChatGPT: 2  Google: 5
  depth: 
    PCFG: 10  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: 

## 4. Simple: vt

In [35]:
id_4_base = [3]
id_4_adversarial = [15, 16, 17, 21, 26]

In [37]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_4 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_4_base)):
        tree = trees[id_4_base[idx]-1]
        base_trees_4[model].append(tree)

In [38]:
base_sent_4 = [base_sent[i-1] for i in id_4_base]

base_metrics_4 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_4:
    base_metrics_4['Sent'].append(sent)

for idx in range(len(base_sent_4)):
    for model in MODELS:
        tree = base_trees_4[model][idx]
        graph = convert_graph(tree)
        base_metrics_4[model].append(network_measures(graph, model))

In [39]:
print("4. Simple: vt \n")
print("base \n")
for idx in range(len(base_sent_4)):
    print(idx)
    print(base_metrics_4['Sent'][idx])
    for metric in list(base_metrics_4['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_4['PCFG'][idx][metric], 
                                                          base_metrics_4['ChatGPT'][idx][metric], 
                                                          base_metrics_4['Google'][idx][metric]))

4. Simple: vt 

base 

0
Arthur rides the horse near the castle .
  num_nodes: 
    PCFG: 37  ChatGPT: 22  Google: 28
  num_edges: 
    PCFG: 36  ChatGPT: 21  Google: 27
  width: 
    PCFG: 3  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 15  ChatGPT: 6  Google: 6
  max_betweenness: 
    PCFG: (15, 0.6523809523809524)  ChatGPT: (5, 0.7428571428571429)  Google: (1, 0.7578347578347578)
  mean_betweenness: 
    PCFG: 0.1885027885027885  ChatGPT: 0.1497835497835498  Google: 0.13583638583638583
  max_degree_centrality: 
    PCFG: (2, 0.08333333333333333)  ChatGPT: (5, 0.19047619047619047)  Google: (1, 0.18518518518518517)
  mean_degree_centrality: 
    PCFG: 0.05405405405405404  ChatGPT: 0.09090909090909091  Google: 0.07142857142857142


In [40]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_4 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_4_adversarial)):
        tree = trees[id_4_adversarial[idx]-1]
        adversarial_trees_4[model].append(tree)

In [41]:
adversarial_sent_4 = [adversarial_sent[i-1] for i in id_4_adversarial]

adversarial_metrics_4 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_4:
    adversarial_metrics_4['Sent'].append(sent)

for idx in range(len(adversarial_sent_4)):
    for model in MODELS:
        tree = adversarial_trees_4[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_4[model].append(network_measures(graph, model))

In [42]:
print("4. Simple: vt \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_4)):
    print(idx)
    print(adversarial_metrics_4['Sent'][idx])
    for metric in list(adversarial_metrics_4['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_4['PCFG'][idx][metric], 
                                                          adversarial_metrics_4['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_4['Google'][idx][metric]))

4. Simple: vt 

adversarial 

0
the master covers the chalice .
  num_nodes: 
    PCFG: 26  ChatGPT: 16  Google: 21
  num_edges: 
    PCFG: 25  ChatGPT: 15  Google: 20
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 10  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: (2, 0.5933333333333334)  ChatGPT: (1, 0.6285714285714287)  Google: (1, 0.7473684210526316)
  mean_betweenness: 
    PCFG: 0.21307692307692308  ChatGPT: 0.18571428571428572  Google: 0.16491228070175437
  max_degree_centrality: 
    PCFG: (2, 0.12)  ChatGPT: (1, 0.2)  Google: (1, 0.2)
  mean_degree_centrality: 
    PCFG: 0.07692307692307693  ChatGPT: 0.125  Google: 0.09523809523809523
1
Guinevere drinks the horse .
  num_nodes: 
    PCFG: 23  ChatGPT: 14  Google: 17
  num_edges: 
    PCFG: 22  ChatGPT: 13  Google: 16
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 10  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: (9, 0.6060606060606061)  ChatGPT: (5, 0.6666666666666666)  Google: (1

## 5. Simple - Passive

In [43]:
id_5_base = [16, 17]

In [44]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_5 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_5_base)):
        tree = trees[id_5_base[idx]-1]
        base_trees_5[model].append(tree)

In [45]:
base_sent_5 = [base_sent[i-1] for i in id_5_base]

base_metrics_5 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_5:
    base_metrics_5['Sent'].append(sent)

for idx in range(len(base_sent_5)):
    for model in MODELS:
        tree = base_trees_5[model][idx]
        graph = convert_graph(tree)
        base_metrics_5[model].append(network_measures(graph, model))

In [46]:
print("5. Simple - Passive \n")
print("base \n")
for idx in range(len(base_sent_5)):
    print(idx)
    print(base_metrics_5['Sent'][idx])
    for metric in list(base_metrics_5['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_5['PCFG'][idx][metric], 
                                                          base_metrics_5['ChatGPT'][idx][metric], 
                                                          base_metrics_5['Google'][idx][metric]))

5. Simple - Passive 

base 

0
the Holy Grail was covered by a yellow fruit .
  num_nodes: 
    PCFG: 36  ChatGPT: 26  Google: 34
  num_edges: 
    PCFG: 35  ChatGPT: 25  Google: 33
  width: 
    PCFG: 3  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 11  ChatGPT: 7  Google: 6
  max_betweenness: 
    PCFG: (12, 0.6420168067226891)  ChatGPT: (9, 0.5866666666666667)  Google: (1, 0.7253787878787878)
  mean_betweenness: 
    PCFG: 0.16591970121381885  ChatGPT: 0.14871794871794874  Google: 0.11692290552584671
  max_degree_centrality: 
    PCFG: (2, 0.08571428571428572)  ChatGPT: (2, 0.16)  Google: (1, 0.15151515151515152)
  mean_degree_centrality: 
    PCFG: 0.055555555555555546  ChatGPT: 0.07692307692307693  Google: 0.058823529411764705
1
Zoot might have been carried by a swallow .
  num_nodes: 
    PCFG: 32  ChatGPT: 24  Google: 30
  num_edges: 
    PCFG: 31  ChatGPT: 23  Google: 29
  width: 
    PCFG: 3  ChatGPT: 3  Google: 6
  depth: 
    PCFG: 10  ChatGPT: 7  Google: 6
  max_betweenness: 
 

## 6. Simple - Apposition

In [47]:
id_6_base = [22, 23]

In [48]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_6 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_6_base)):
        tree = trees[id_6_base[idx]-1]
        base_trees_6[model].append(tree)

In [49]:
base_sent_6 = [base_sent[i-1] for i in id_6_base]

base_metrics_6 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_6:
    base_metrics_6['Sent'].append(sent)

for idx in range(len(base_sent_6)):
    for model in MODELS:
        tree = base_trees_6[model][idx]
        graph = convert_graph(tree)
        base_metrics_6[model].append(network_measures(graph, model))

In [50]:
print("6. Simple - Apposition \n")
print("base \n")
for idx in range(len(base_sent_6)):
    print(idx)
    print(base_metrics_6['Sent'][idx])
    for metric in list(base_metrics_6['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_6['PCFG'][idx][metric], 
                                                          base_metrics_6['ChatGPT'][idx][metric], 
                                                          base_metrics_6['Google'][idx][metric]))

6. Simple - Apposition 

base 

0
Arthur , sixty inches , is a tiny king .
  num_nodes: 
    PCFG: 37  ChatGPT: 22  Google: 34
  num_edges: 
    PCFG: 36  ChatGPT: 21  Google: 33
  width: 
    PCFG: 4  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 9  ChatGPT: 5  Google: 6
  max_betweenness: 
    PCFG: (2, 0.6126984126984127)  ChatGPT: (1, 0.6095238095238096)  Google: (2, 0.6856060606060607)
  mean_betweenness: 
    PCFG: 0.16542256542256545  ChatGPT: 0.14285714285714288  Google: 0.11458333333333336
  max_degree_centrality: 
    PCFG: (2, 0.1111111111111111)  ChatGPT: (2, 0.23809523809523808)  Google: (2, 0.15151515151515152)
  mean_degree_centrality: 
    PCFG: 0.054054054054054036  ChatGPT: 0.09090909090909093  Google: 0.058823529411764705
1
Arthur knows Patsy , the trusty servant .
  num_nodes: 
    PCFG: 35  ChatGPT: 20  Google: 27
  num_edges: 
    PCFG: 34  ChatGPT: 19  Google: 26
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 14  ChatGPT: 5  Google: 6
  max_betweenn

## 7. Compound

In [51]:
id_7_base = [14, 15, 18]
id_7_adversarial = [2, 7, 13, 14, 62, 70, 75, 76, 78, 82, 87]

In [52]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_7 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_7_base)):
        tree = trees[id_7_base[idx]-1]
        base_trees_7[model].append(tree)

In [53]:
base_sent_7 = [base_sent[i-1] for i in id_7_base]

base_metrics_7 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_7:
    base_metrics_7['Sent'].append(sent)

for idx in range(len(base_sent_7)):
    for model in MODELS:
        tree = base_trees_7[model][idx]
        graph = convert_graph(tree)
        base_metrics_7[model].append(network_measures(graph, model))

In [54]:
print("7. Compound \n")
print("base \n")
for idx in range(len(base_sent_7)):
    print(idx)
    print(base_metrics_7['Sent'][idx])
    for metric in list(base_metrics_7['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_7['PCFG'][idx][metric], 
                                                          base_metrics_7['ChatGPT'][idx][metric], 
                                                          base_metrics_7['Google'][idx][metric]))

7. Compound 

base 

0
either Arthur knows or Patsy does .
  num_nodes: 
    PCFG: 25  ChatGPT: 19  Google: 23
  num_edges: 
    PCFG: 24  ChatGPT: 18  Google: 22
  width: 
    PCFG: 5  ChatGPT: 3  Google: 5
  depth: 
    PCFG: 7  ChatGPT: 5  Google: 5
  max_betweenness: 
    PCFG: (2, 0.6956521739130435)  ChatGPT: (7, 0.7647058823529412)  Google: (1, 0.8441558441558441)
  mean_betweenness: 
    PCFG: 0.17014492753623187  ChatGPT: 0.1537667698658411  Google: 0.13683418031244118
  max_degree_centrality: 
    PCFG: (2, 0.20833333333333331)  ChatGPT: (7, 0.2777777777777778)  Google: (1, 0.2727272727272727)
  mean_degree_centrality: 
    PCFG: 0.07999999999999999  ChatGPT: 0.10526315789473684  Google: 0.08695652173913042
1
neither Sir Lancelot nor Guinevere will speak of it .
  num_nodes: 
    PCFG: 35  ChatGPT: 26  Google: 33
  num_edges: 
    PCFG: 34  ChatGPT: 25  Google: 32
  width: 
    PCFG: 3  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 8  ChatGPT: 7  Google: 5
  max_betweenness: 
   

In [55]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_7 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_7_adversarial)):
        tree = trees[id_7_adversarial[idx]-1]
        adversarial_trees_7[model].append(tree)

In [56]:
adversarial_sent_7 = [adversarial_sent[i-1] for i in id_7_adversarial]

adversarial_metrics_7 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_7:
    adversarial_metrics_7['Sent'].append(sent)

for idx in range(len(adversarial_sent_7)):
    for model in MODELS:
        tree = adversarial_trees_7[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_7[model].append(network_measures(graph, model))

In [57]:
print("7. Compound \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_7)):
    print(idx)
    print(adversarial_metrics_7['Sent'][idx])
    for metric in list(adversarial_metrics_7['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_7['PCFG'][idx][metric], 
                                                          adversarial_metrics_7['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_7['Google'][idx][metric]))

7. Compound 

adversarial 

0
neither no king knows nor no chalice does !
  num_nodes: 
    PCFG: 31  ChatGPT: 20  Google: 31
  num_edges: 
    PCFG: 30  ChatGPT: 19  Google: 30
  width: 
    PCFG: 5  ChatGPT: 1  Google: 4
  depth: 
    PCFG: 7  ChatGPT: 4  Google: 6
  max_betweenness: 
    PCFG: (2, 0.6620689655172414)  ChatGPT: (2, 0.9473684210526315)  Google: (1, 0.7494252873563219)
  mean_betweenness: 
    PCFG: 0.15372636262513903  ChatGPT: 0.09473684210526315  Google: 0.1246570263255469
  max_degree_centrality: 
    PCFG: (2, 0.16666666666666666)  ChatGPT: (2, 0.5263157894736842)  Google: (1, 0.16666666666666666)
  mean_degree_centrality: 
    PCFG: 0.06451612903225808  ChatGPT: 0.09999999999999999  Google: 0.06451612903225808
1
neither a defeater knows nor Camelot does !
  num_nodes: 
    PCFG: 28  ChatGPT: 18  Google: 27
  num_edges: 
    PCFG: 27  ChatGPT: 17  Google: 26
  width: 
    PCFG: 5  ChatGPT: 1  Google: 4
  depth: 
    PCFG: 7  ChatGPT: 4  Google: 5
  max_betweenness

## 8. Complex - Cleft

In [58]:
id_8_base = [13]
id_8_adversarial = [3, 12, 79, 80, 83, 85, 88]

In [59]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_8 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_8_base)):
        tree = trees[id_8_base[idx]-1]
        base_trees_8[model].append(tree)

In [60]:
base_sent_8 = [base_sent[i-1] for i in id_8_base]

base_metrics_8 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_8:
    base_metrics_8['Sent'].append(sent)

for idx in range(len(base_sent_8)):
    for model in MODELS:
        tree = base_trees_8[model][idx]
        graph = convert_graph(tree)
        base_metrics_8[model].append(network_measures(graph, model))

In [61]:
print("8. Complex - Cleft \n")
print("base \n")
for idx in range(len(base_sent_8)):
    print(idx)
    print(base_metrics_8['Sent'][idx])
    for metric in list(base_metrics_8['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_8['PCFG'][idx][metric], 
                                                          base_metrics_8['ChatGPT'][idx][metric], 
                                                          base_metrics_8['Google'][idx][metric]))

8. Complex - Cleft 

base 

0
it is Sir Lancelot who knows Zoot !
  num_nodes: 
    PCFG: 29  ChatGPT: 26  Google: 27
  num_edges: 
    PCFG: 28  ChatGPT: 25  Google: 26
  width: 
    PCFG: 4  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 15  ChatGPT: 9  Google: 6
  max_betweenness: 
    PCFG: (8, 0.6375661375661376)  ChatGPT: (8, 0.6633333333333333)  Google: (8, 0.7046153846153846)
  mean_betweenness: 
    PCFG: 0.20507206714103263  ChatGPT: 0.16179487179487181  Google: 0.1336752136752137
  max_degree_centrality: 
    PCFG: (2, 0.14285714285714285)  ChatGPT: (1, 0.12)  Google: (1, 0.15384615384615385)
  mean_degree_centrality: 
    PCFG: 0.06896551724137931  ChatGPT: 0.07692307692307693  Google: 0.07407407407407407


In [62]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_8 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_8_adversarial)):
        tree = trees[id_8_adversarial[idx]-1]
        adversarial_trees_8[model].append(tree)

In [63]:
adversarial_sent_8 = [adversarial_sent[i-1] for i in id_8_adversarial]

adversarial_metrics_8 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_8:
    adversarial_metrics_8['Sent'].append(sent)

for idx in range(len(adversarial_sent_8)):
    for model in MODELS:
        tree = adversarial_trees_8[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_8[model].append(network_measures(graph, model))

In [64]:
print("8. Complex - Cleft \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_8)):
    print(idx)
    print(adversarial_metrics_8['Sent'][idx])
    for metric in list(adversarial_metrics_8['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_8['PCFG'][idx][metric], 
                                                          adversarial_metrics_8['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_8['Google'][idx][metric]))

8. Complex - Cleft 

adversarial 

0
it is Sir Bedevere who knows every king !
  num_nodes: 
    PCFG: 32  ChatGPT: 28  Google: 31
  num_edges: 
    PCFG: 31  ChatGPT: 27  Google: 30
  width: 
    PCFG: 4  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 15  ChatGPT: 9  Google: 6
  max_betweenness: 
    PCFG: (8, 0.6086021505376344)  ChatGPT: (8, 0.6467236467236468)  Google: (1, 0.7218390804597701)
  mean_betweenness: 
    PCFG: 0.1989247311827957  ChatGPT: 0.15689865689865692  Google: 0.1228772710418984
  max_degree_centrality: 
    PCFG: (2, 0.12903225806451613)  ChatGPT: (1, 0.1111111111111111)  Google: (1, 0.16666666666666666)
  mean_degree_centrality: 
    PCFG: 0.0625  ChatGPT: 0.07142857142857142  Google: 0.06451612903225808
1
it is Arthur that knows Camelot .
  num_nodes: 
    PCFG: 28  ChatGPT: 21  Google: 24
  num_edges: 
    PCFG: 27  ChatGPT: 20  Google: 23
  width: 
    PCFG: 4  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 15  ChatGPT: 7  Google: 6
  max_betweenness: 
    PCFG: (8, 

## 9. Complex - Relative Clause

In [65]:
id_9_base = [27]

In [66]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_9 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_9_base)):
        tree = trees[id_9_base[idx]-1]
        base_trees_9[model].append(tree)

In [67]:
base_sent_9 = [base_sent[i-1] for i in id_9_base]

base_metrics_9 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_9:
    base_metrics_9['Sent'].append(sent)

for idx in range(len(base_sent_9)):
    for model in MODELS:
        tree = base_trees_9[model][idx]
        graph = convert_graph(tree)
        base_metrics_9[model].append(network_measures(graph, model))

In [69]:
print("9. Complex - Relative Clause \n")
print("base \n")
for idx in range(len(base_sent_9)):
    print(idx)
    print(base_metrics_9['Sent'][idx])
    for metric in list(base_metrics_9['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_9['PCFG'][idx][metric], 
                                                          base_metrics_9['ChatGPT'][idx][metric], 
                                                          base_metrics_9['Google'][idx][metric]))

9. Complex - Relative Clause 

base 

0
the king drank to the castle that was his home .
  num_nodes: 
    PCFG: 45  ChatGPT: 32  Google: 39
  num_edges: 
    PCFG: 44  ChatGPT: 31  Google: 38
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 16  ChatGPT: 9  Google: 8
  max_betweenness: 
    PCFG: (20, 0.6289640591966174)  ChatGPT: (10, 0.6903225806451613)  Google: (16, 0.6443812233285918)
  mean_betweenness: 
    PCFG: 0.16894526661968523  ChatGPT: 0.1388440860215054  Google: 0.11689827479301165
  max_degree_centrality: 
    PCFG: (12, 0.09090909090909091)  ChatGPT: (10, 0.12903225806451613)  Google: (1, 0.10526315789473684)
  mean_degree_centrality: 
    PCFG: 0.04444444444444444  ChatGPT: 0.0625  Google: 0.051282051282051294


## 10. Complex - Adverbial Clause

In [70]:
id_10_base = [19, 28]

In [71]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_10 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_10_base)):
        tree = trees[id_10_base[idx]-1]
        base_trees_10[model].append(tree)

In [72]:
base_sent_10 = [base_sent[i-1] for i in id_10_base]

base_metrics_10 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_10:
    base_metrics_10['Sent'].append(sent)

for idx in range(len(base_sent_10)):
    for model in MODELS:
        tree = base_trees_10[model][idx]
        graph = convert_graph(tree)
        base_metrics_10[model].append(network_measures(graph, model))

In [73]:
print("10. Complex - Adverbial Clause \n")
print("base \n")
for idx in range(len(base_sent_10)):
    print(idx)
    print(base_metrics_10['Sent'][idx])
    for metric in list(base_metrics_10['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_10['PCFG'][idx][metric], 
                                                          base_metrics_10['ChatGPT'][idx][metric], 
                                                          base_metrics_10['Google'][idx][metric]))

10. Complex - Adverbial Clause 

base 

0
they migrate precisely because they know they will grow .
  num_nodes: 
    PCFG: 37  ChatGPT: 29  Google: 33
  num_edges: 
    PCFG: 36  ChatGPT: 28  Google: 32
  width: 
    PCFG: 4  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 9  ChatGPT: 10  Google: 6
  max_betweenness: 
    PCFG: (2, 0.6047619047619047)  ChatGPT: (14, 0.611111111111111)  Google: (11, 0.7338709677419355)
  mean_betweenness: 
    PCFG: 0.14199914199914201  ChatGPT: 0.15197956577266922  Google: 0.1100928641251222
  max_degree_centrality: 
    PCFG: (2, 0.1111111111111111)  ChatGPT: (1, 0.10714285714285714)  Google: (1, 0.15625)
  mean_degree_centrality: 
    PCFG: 0.05405405405405404  ChatGPT: 0.06896551724137931  Google: 0.06060606060606061
1
when the king drinks , Patsy drinks .
  num_nodes: 
    PCFG: 32  ChatGPT: 23  Google: 26
  num_edges: 
    PCFG: 31  ChatGPT: 22  Google: 25
  width: 
    PCFG: 4  ChatGPT: 5  Google: 6
  depth: 
    PCFG: 8  ChatGPT: 6  Google: 5
  max_b

## 11. Complex - Object Clause

In [74]:
id_11_base = [25, 26]
id_11_adversarial = [63, 71, 73]

In [75]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_11 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_11_base)):
        tree = trees[id_11_base[idx]-1]
        base_trees_11[model].append(tree)

In [76]:
base_sent_11 = [base_sent[i-1] for i in id_11_base]

base_metrics_11 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_11:
    base_metrics_11['Sent'].append(sent)

for idx in range(len(base_sent_11)):
    for model in MODELS:
        tree = base_trees_11[model][idx]
        graph = convert_graph(tree)
        base_metrics_11[model].append(network_measures(graph, model))

In [77]:
print("11. Complex - Object Clause \n")
print("base \n")
for idx in range(len(base_sent_11)):
    print(idx)
    print(base_metrics_11['Sent'][idx])
    for metric in list(base_metrics_11['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_11['PCFG'][idx][metric], 
                                                          base_metrics_11['ChatGPT'][idx][metric], 
                                                          base_metrics_11['Google'][idx][metric]))

11. Complex - Object Clause 

base 

0
he knows what they are covering with that story .
  num_nodes: 
    PCFG: 35  ChatGPT: 31  Google: 34
  num_edges: 
    PCFG: 34  ChatGPT: 30  Google: 33
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 11  ChatGPT: 10  Google: 7
  max_betweenness: 
    PCFG: (10, 0.7023172905525846)  ChatGPT: (12, 0.6045977011494252)  Google: (8, 0.7878787878787878)
  mean_betweenness: 
    PCFG: 0.1391902215431627  ChatGPT: 0.14987022617723397  Google: 0.11257798573975045
  max_degree_centrality: 
    PCFG: (10, 0.14705882352941177)  ChatGPT: (1, 0.1)  Google: (8, 0.18181818181818182)
  mean_degree_centrality: 
    PCFG: 0.05714285714285713  ChatGPT: 0.06451612903225808  Google: 0.058823529411764705
1
Arthur suggested that the castle be carried .
  num_nodes: 
    PCFG: 31  ChatGPT: 24  Google: 27
  num_edges: 
    PCFG: 30  ChatGPT: 23  Google: 26
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 11  ChatGPT: 8  Google: 6
  max_be

In [78]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_11 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_11_adversarial)):
        tree = trees[id_11_adversarial[idx]-1]
        adversarial_trees_11[model].append(tree)

In [79]:
adversarial_sent_11 = [adversarial_sent[i-1] for i in id_11_adversarial]

adversarial_metrics_11 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_11:
    adversarial_metrics_11['Sent'].append(sent)

for idx in range(len(adversarial_sent_11)):
    for model in MODELS:
        tree = adversarial_trees_11[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_11[model].append(network_measures(graph, model))

In [80]:
print("11. Complex - Object Clause \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_11)):
    print(idx)
    print(adversarial_metrics_11['Sent'][idx])
    for metric in list(adversarial_metrics_11['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_11['PCFG'][idx][metric], 
                                                          adversarial_metrics_11['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_11['Google'][idx][metric]))

11. Complex - Object Clause 

adversarial 

0
this fruit ought to drink what has the home !
  num_nodes: 
    PCFG: 37  ChatGPT: 27  Google: 35
  num_edges: 
    PCFG: 36  ChatGPT: 26  Google: 34
  width: 
    PCFG: 3  ChatGPT: 3  Google: 3
  depth: 
    PCFG: 13  ChatGPT: 8  Google: 7
  max_betweenness: 
    PCFG: (11, 0.5841269841269842)  ChatGPT: (10, 0.6553846153846153)  Google: (12, 0.6648841354723707)
  mean_betweenness: 
    PCFG: 0.17237237237237238  ChatGPT: 0.14643874643874644  Google: 0.1194295900178253
  max_degree_centrality: 
    PCFG: (2, 0.08333333333333333)  ChatGPT: (10, 0.15384615384615385)  Google: (1, 0.11764705882352941)
  mean_degree_centrality: 
    PCFG: 0.05405405405405404  ChatGPT: 0.07407407407407407  Google: 0.05714285714285713
1
every land goes who drinks through any swallow .
  num_nodes: 
    PCFG: 37  ChatGPT: 27  Google: 32
  num_edges: 
    PCFG: 36  ChatGPT: 26  Google: 31
  width: 
    PCFG: 3  ChatGPT: 2  Google: 3
  depth: 
    PCFG: 10  ChatGPT: 

## 12. Imperative

In [81]:
id_12_base = [20]

In [82]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_12 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_12_base)):
        tree = trees[id_12_base[idx]-1]
        base_trees_12[model].append(tree)

In [83]:
base_sent_12 = [base_sent[i-1] for i in id_12_base]

base_metrics_12 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_12:
    base_metrics_12['Sent'].append(sent)

for idx in range(len(base_sent_12)):
    for model in MODELS:
        tree = base_trees_12[model][idx]
        graph = convert_graph(tree)
        base_metrics_12[model].append(network_measures(graph, model))

In [84]:
print("12. Imperative \n")
print("base \n")
for idx in range(len(base_sent_12)):
    print(idx)
    print(base_metrics_12['Sent'][idx])
    for metric in list(base_metrics_12['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_12['PCFG'][idx][metric], 
                                                          base_metrics_12['ChatGPT'][idx][metric], 
                                                          base_metrics_12['Google'][idx][metric]))

12. Imperative 

base 

0
do not speak !
  num_nodes: 
    PCFG: 11  ChatGPT: 11  Google: 13
  num_edges: 
    PCFG: 10  ChatGPT: 10  Google: 12
  width: 
    PCFG: 4  ChatGPT: 1  Google: 3
  depth: 
    PCFG: 5  ChatGPT: 5  Google: 4
  max_betweenness: 
    PCFG: (2, 0.7777777777777778)  ChatGPT: (2, 0.7333333333333334)  Google: (1, 0.8181818181818182)
  mean_betweenness: 
    PCFG: 0.23232323232323235  ChatGPT: 0.19595959595959594  Google: 0.20745920745920748
  max_degree_centrality: 
    PCFG: (2, 0.4)  ChatGPT: (2, 0.4)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.18181818181818185  ChatGPT: 0.18181818181818185  Google: 0.15384615384615383


## 13. Interrogative - Yes/No

In [85]:
id_13_base = [5, 9]
id_13_adversarial = [4, 29, 33, 36, 37, 38, 39, 40, 41, 44, 45, 49, 50, 51, 54, 55, 74]

In [86]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_13 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_13_base)):
        tree = trees[id_13_base[idx]-1]
        base_trees_13[model].append(tree)

In [87]:
base_sent_13 = [base_sent[i-1] for i in id_13_base]

base_metrics_13 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_13:
    base_metrics_13['Sent'].append(sent)

for idx in range(len(base_sent_13)):
    for model in MODELS:
        tree = base_trees_13[model][idx]
        graph = convert_graph(tree)
        base_metrics_13[model].append(network_measures(graph, model))

In [88]:
print("13. Interrogative - Yes/No \n")
print("base \n")
for idx in range(len(base_sent_13)):
    print(idx)
    print(base_metrics_13['Sent'][idx])
    for metric in list(base_metrics_13['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_13['PCFG'][idx][metric], 
                                                          base_metrics_13['ChatGPT'][idx][metric], 
                                                          base_metrics_13['Google'][idx][metric]))

13. Interrogative - Yes/No 

base 

0
do coconuts speak ?
  num_nodes: 
    PCFG: 13  ChatGPT: 11  Google: 13
  num_edges: 
    PCFG: 12  ChatGPT: 10  Google: 12
  width: 
    PCFG: 4  ChatGPT: 4  Google: 3
  depth: 
    PCFG: 6  ChatGPT: 4  Google: 4
  max_betweenness: 
    PCFG: (2, 0.7727272727272727)  ChatGPT: (1, 0.8222222222222223)  Google: (1, 0.8181818181818182)
  mean_betweenness: 
    PCFG: 0.2261072261072261  ChatGPT: 0.21212121212121213  Google: 0.20745920745920748
  max_degree_centrality: 
    PCFG: (2, 0.3333333333333333)  ChatGPT: (1, 0.4)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.15384615384615383  ChatGPT: 0.18181818181818185  Google: 0.15384615384615383
1
are they suggesting Arthur ride to Camelot ?
  num_nodes: 
    PCFG: 35  ChatGPT: 22  Google: 27
  num_edges: 
    PCFG: 34  ChatGPT: 21  Google: 26
  width: 
    PCFG: 4  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 12  ChatGPT: 8  Google: 6
  max_betweenness: 
    PCFG: (9, 0.668449197860

In [89]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_13 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_13_adversarial)):
        tree = trees[id_13_adversarial[idx]-1]
        adversarial_trees_13[model].append(tree)

In [90]:
adversarial_sent_13 = [adversarial_sent[i-1] for i in id_13_adversarial]

adversarial_metrics_13 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_13:
    adversarial_metrics_13['Sent'].append(sent)

for idx in range(len(adversarial_sent_13)):
    for model in MODELS:
        tree = adversarial_trees_13[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_13[model].append(network_measures(graph, model))

In [91]:
print("13. Interrogative - Yes/No \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_13)):
    print(idx)
    print(adversarial_metrics_13['Sent'][idx])
    for metric in list(adversarial_metrics_13['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_13['PCFG'][idx][metric], 
                                                          adversarial_metrics_13['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_13['Google'][idx][metric]))

13. Interrogative - Yes/No 

adversarial 

0
do halves ride ?
  num_nodes: 
    PCFG: 13  ChatGPT: 12  Google: 13
  num_edges: 
    PCFG: 12  ChatGPT: 11  Google: 12
  width: 
    PCFG: 4  ChatGPT: 1  Google: 3
  depth: 
    PCFG: 6  ChatGPT: 5  Google: 4
  max_betweenness: 
    PCFG: (2, 0.7727272727272727)  ChatGPT: (2, 0.7454545454545454)  Google: (1, 0.8181818181818182)
  mean_betweenness: 
    PCFG: 0.2261072261072261  ChatGPT: 0.1984848484848485  Google: 0.20745920745920748
  max_degree_centrality: 
    PCFG: (2, 0.3333333333333333)  ChatGPT: (2, 0.36363636363636365)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.15384615384615383  ChatGPT: 0.16666666666666663  Google: 0.15384615384615383
1
are areas drinking ?
  num_nodes: 
    PCFG: 14  ChatGPT: 10  Google: 14
  num_edges: 
    PCFG: 13  ChatGPT: 9  Google: 13
  width: 
    PCFG: 4  ChatGPT: 1  Google: 2
  depth: 
    PCFG: 6  ChatGPT: 4  Google: 5
  max_betweenness: 
    PCFG: (2, 0.782051282051282)  C

## 14. Interrogative - Wh-Questions

In [92]:
id_14_base = [6, 8]
id_14_adversarial = [9, 30, 31, 64, 65, 67, 68, 81]

In [93]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_14 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_14_base)):
        tree = trees[id_14_base[idx]-1]
        base_trees_14[model].append(tree)

In [94]:
base_sent_14 = [base_sent[i-1] for i in id_14_base]

base_metrics_14 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_14:
    base_metrics_14['Sent'].append(sent)

for idx in range(len(base_sent_14)):
    for model in MODELS:
        tree = base_trees_14[model][idx]
        graph = convert_graph(tree)
        base_metrics_14[model].append(network_measures(graph, model))

In [95]:
print("14. Interrogative - Wh-Questions \n")
print("base \n")
for idx in range(len(base_sent_14)):
    print(idx)
    print(base_metrics_14['Sent'][idx])
    for metric in list(base_metrics_14['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_14['PCFG'][idx][metric], 
                                                          base_metrics_14['ChatGPT'][idx][metric], 
                                                          base_metrics_14['Google'][idx][metric]))

14. Interrogative - Wh-Questions 

base 

0
what does Arthur ride ?
  num_nodes: 
    PCFG: 15  ChatGPT: 13  Google: 16
  num_edges: 
    PCFG: 14  ChatGPT: 12  Google: 15
  width: 
    PCFG: 5  ChatGPT: 3  Google: 4
  depth: 
    PCFG: 6  ChatGPT: 5  Google: 4
  max_betweenness: 
    PCFG: (2, 0.8241758241758242)  ChatGPT: (4, 0.7727272727272727)  Google: (1, 0.8571428571428572)
  mean_betweenness: 
    PCFG: 0.19120879120879122  ChatGPT: 0.1888111888111888  Google: 0.17261904761904762
  max_degree_centrality: 
    PCFG: (2, 0.3571428571428571)  ChatGPT: (4, 0.3333333333333333)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.1333333333333333  ChatGPT: 0.1538461538461538  Google: 0.125
1
why does England have a king ?
  num_nodes: 
    PCFG: 25  ChatGPT: 20  Google: 23
  num_edges: 
    PCFG: 24  ChatGPT: 19  Google: 22
  width: 
    PCFG: 5  ChatGPT: 3  Google: 5
  depth: 
    PCFG: 9  ChatGPT: 6  Google: 5
  max_betweenness: 
    PCFG: (2, 0.6702898550724637) 

In [96]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_14 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_14_adversarial)):
        tree = trees[id_14_adversarial[idx]-1]
        adversarial_trees_14[model].append(tree)

In [97]:
adversarial_sent_14 = [adversarial_sent[i-1] for i in id_14_adversarial]

adversarial_metrics_14 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_14:
    adversarial_metrics_14['Sent'].append(sent)

for idx in range(len(adversarial_sent_14)):
    for model in MODELS:
        tree = adversarial_trees_14[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_14[model].append(network_measures(graph, model))

In [98]:
print("14. Interrogative - Wh-Questions \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_14)):
    print(idx)
    print(adversarial_metrics_14['Sent'][idx])
    for metric in list(adversarial_metrics_14['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_14['PCFG'][idx][metric], 
                                                          adversarial_metrics_14['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_14['Google'][idx][metric]))

14. Interrogative - Wh-Questions 

adversarial 

0
who does England have ?
  num_nodes: 
    PCFG: 15  ChatGPT: 12  Google: 16
  num_edges: 
    PCFG: 14  ChatGPT: 11  Google: 15
  width: 
    PCFG: 5  ChatGPT: 2  Google: 4
  depth: 
    PCFG: 6  ChatGPT: 4  Google: 4
  max_betweenness: 
    PCFG: (2, 0.8241758241758242)  ChatGPT: (5, 0.8363636363636363)  Google: (1, 0.8571428571428572)
  mean_betweenness: 
    PCFG: 0.19120879120879122  ChatGPT: 0.19393939393939394  Google: 0.17261904761904762
  max_degree_centrality: 
    PCFG: (2, 0.3571428571428571)  ChatGPT: (5, 0.4545454545454546)  Google: (1, 0.3333333333333333)
  mean_degree_centrality: 
    PCFG: 0.1333333333333333  ChatGPT: 0.16666666666666663  Google: 0.125
1
what do they know ?
  num_nodes: 
    PCFG: 13  ChatGPT: 13  Google: 16
  num_edges: 
    PCFG: 12  ChatGPT: 12  Google: 15
  width: 
    PCFG: 5  ChatGPT: 2  Google: 4
  depth: 
    PCFG: 4  ChatGPT: 4  Google: 4
  max_betweenness: 
    PCFG: (2, 0.8636363636363636)  C

## 15. Interrogative - Wh-Questions with Object Clause

In [99]:
id_15_base = [7]
id_15_adversarial = [8, 32, 35, 72]

In [100]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

base_trees_15 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_Base_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_15_base)):
        tree = trees[id_15_base[idx]-1]
        base_trees_15[model].append(tree)

In [101]:
base_sent_15 = [base_sent[i-1] for i in id_15_base]

base_metrics_15 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in base_sent_15:
    base_metrics_15['Sent'].append(sent)

for idx in range(len(base_sent_15)):
    for model in MODELS:
        tree = base_trees_15[model][idx]
        graph = convert_graph(tree)
        base_metrics_15[model].append(network_measures(graph, model))

In [102]:
print("15. Interrogative - Wh-Questions with Object Clause \n")
print("base \n")
for idx in range(len(base_sent_15)):
    print(idx)
    print(base_metrics_15['Sent'][idx])
    for metric in list(base_metrics_15['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(base_metrics_15['PCFG'][idx][metric], 
                                                          base_metrics_15['ChatGPT'][idx][metric], 
                                                          base_metrics_15['Google'][idx][metric]))

15. Interrogative - Wh-Questions with Object Clause 

base 

0
who does Arthur suggest she carry ?
  num_nodes: 
    PCFG: 25  ChatGPT: 19  Google: 23
  num_edges: 
    PCFG: 24  ChatGPT: 18  Google: 22
  width: 
    PCFG: 5  ChatGPT: 3  Google: 5
  depth: 
    PCFG: 8  ChatGPT: 6  Google: 5
  max_betweenness: 
    PCFG: (2, 0.6702898550724637)  ChatGPT: (4, 0.7254901960784313)  Google: (1, 0.8441558441558441)
  mean_betweenness: 
    PCFG: 0.16608695652173913  ChatGPT: 0.1537667698658411  Google: 0.13683418031244118
  max_degree_centrality: 
    PCFG: (2, 0.20833333333333331)  ChatGPT: (4, 0.2222222222222222)  Google: (1, 0.2727272727272727)
  mean_degree_centrality: 
    PCFG: 0.07999999999999999  ChatGPT: 0.10526315789473684  Google: 0.08695652173913042


In [103]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_15 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_15_adversarial)):
        tree = trees[id_15_adversarial[idx]-1]
        adversarial_trees_15[model].append(tree)

In [104]:
adversarial_sent_15 = [adversarial_sent[i-1] for i in id_15_adversarial]

adversarial_metrics_15 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_15:
    adversarial_metrics_15['Sent'].append(sent)

for idx in range(len(adversarial_sent_15)):
    for model in MODELS:
        tree = adversarial_trees_15[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_15[model].append(network_measures(graph, model))

In [105]:
print("15. Interrogative - Wh-Questions with Object Clause \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_15)):
    print(idx)
    print(adversarial_metrics_15['Sent'][idx])
    for metric in list(adversarial_metrics_15['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_15['PCFG'][idx][metric], 
                                                          adversarial_metrics_15['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_15['Google'][idx][metric]))

15. Interrogative - Wh-Questions with Object Clause 

adversarial 

0
what does England suggest Patsy suggest ?
  num_nodes: 
    PCFG: 27  ChatGPT: 20  Google: 23
  num_edges: 
    PCFG: 26  ChatGPT: 19  Google: 22
  width: 
    PCFG: 5  ChatGPT: 2  Google: 5
  depth: 
    PCFG: 10  ChatGPT: 6  Google: 5
  max_betweenness: 
    PCFG: (2, 0.6369230769230769)  ChatGPT: (10, 0.7076023391812865)  Google: (1, 0.8441558441558441)
  mean_betweenness: 
    PCFG: 0.1731054131054131  ChatGPT: 0.15643274853801167  Google: 0.13683418031244118
  max_degree_centrality: 
    PCFG: (2, 0.19230769230769232)  ChatGPT: (5, 0.21052631578947367)  Google: (1, 0.2727272727272727)
  mean_degree_centrality: 
    PCFG: 0.07407407407407407  ChatGPT: 0.09999999999999998  Google: 0.08695652173913042
1
who does one suggest the coconut cover ?
  num_nodes: 
    PCFG: 30  ChatGPT: 19  Google: 26
  num_edges: 
    PCFG: 29  ChatGPT: 18  Google: 25
  width: 
    PCFG: 5  ChatGPT: 2  Google: 5
  depth: 
    PCFG: 10  C

## 16. Interrogative - Rhetorical

In [106]:
id_16_adversarial = [18, 23]

In [107]:
MODELS = ['PCFG', 'ChatGPT', 'Google']

adversarial_trees_16 = {'PCFG':[], 'ChatGPT':[], 'Google':[]}

for i in range(3):
    model = MODELS[i]
    with open(save_dir+'\\Trees\\'+model+'_adversarial_Trees.pkl', 'rb') as f:
        trees = pickle.load(f)
        
    for idx in range(len(id_16_adversarial)):
        tree = trees[id_16_adversarial[idx]-1]
        adversarial_trees_16[model].append(tree)

In [108]:
adversarial_sent_16 = [adversarial_sent[i-1] for i in id_16_adversarial]

adversarial_metrics_16 = {'Sent':[], 'PCFG':[], 'ChatGPT':[], 'Google':[]}

for sent in adversarial_sent_16:
    adversarial_metrics_16['Sent'].append(sent)

for idx in range(len(adversarial_sent_16)):
    for model in MODELS:
        tree = adversarial_trees_16[model][idx]
        graph = convert_graph(tree)
        adversarial_metrics_16[model].append(network_measures(graph, model))

In [109]:
print("16. Interrogative - Rhetorical \n")
print("adversarial \n")
for idx in range(len(adversarial_sent_16)):
    print(idx)
    print(adversarial_metrics_16['Sent'][idx])
    for metric in list(adversarial_metrics_16['PCFG'][0].keys()):
        print("  {}: ".format(metric))
        print("    PCFG: {}  ChatGPT: {}  Google: {}".format(adversarial_metrics_16['PCFG'][idx][metric], 
                                                          adversarial_metrics_16['ChatGPT'][idx][metric], 
                                                          adversarial_metrics_16['Google'][idx][metric]))

16. Interrogative - Rhetorical 

adversarial 

0
this husk goes ?
  num_nodes: 
    PCFG: 15  ChatGPT: 11  Google: 14
  num_edges: 
    PCFG: 14  ChatGPT: 10  Google: 13
  width: 
    PCFG: 3  ChatGPT: 2  Google: 2
  depth: 
    PCFG: 6  ChatGPT: 4  Google: 5
  max_betweenness: 
    PCFG: (4, 0.6263736263736264)  ChatGPT: (2, 0.6222222222222222)  Google: (1, 0.6538461538461539)
  mean_betweenness: 
    PCFG: 0.23369963369963373  ChatGPT: 0.23636363636363636  Google: 0.21794871794871792
  max_degree_centrality: 
    PCFG: (2, 0.21428571428571427)  ChatGPT: (2, 0.30000000000000004)  Google: (1, 0.23076923076923078)
  mean_degree_centrality: 
    PCFG: 0.1333333333333333  ChatGPT: 0.18181818181818185  Google: 0.14285714285714282
1
a pound carries a story ?
  num_nodes: 
    PCFG: 25  ChatGPT: 15  Google: 21
  num_edges: 
    PCFG: 24  ChatGPT: 14  Google: 20
  width: 
    PCFG: 3  ChatGPT: 2  Google: 3
  depth: 
    PCFG: 10  ChatGPT: 4  Google: 5
  max_betweenness: 
    PCFG: (12, 0.5942